In [ ]:
#install.packages(c("ggplot2", "car", "lmtest", "MASS", "nortest", "glmnet", "caret"))
library(ggplot2)# Para gráficos
library(car)# Para diagnósticos de regresión (VIF)
library(lmtest)# Para tests de diagnóstico (bptest)
library(MASS)# Para regresión robusta (rlm)
library(nortest)# Para tests de normalidad
library(glmnet)# Para regresión penalizada (Ridge, Lasso, Elastic Net)
library(caret)# Para facilitar el modelado y la evaluación

Installing packages into ‘/home/alvarezg/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)

Warning message:
“package ‘MASS’ is not available for this version of R
‘MASS’ version 7.3-65 is in the repositories but depends on R (>= 4.4.0)
‘MASS’ version 7.3-65 is in the repositories but depends on R (>= 4.6)

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Loading required package: carData

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: Matrix

Loaded glmnet 4.1-10

Loading required package: lattice

